![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-large.png)  
[National Science Data Fabric](https://nationalsciencedatafabric.org/) 

# Converting TIFF  data 


In [ ]:
import os,sys,glob,argparse,time
import numpy as np
from matplotlib import pyplot as plt
from skimage import io

import os,sys

# sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
# sys.path.append(r"C:\projects\openvisuspy\src")

import OpenVisus as ov
import logging
os.environ["VISUS_DISABLE_WRITE_LOCK"]="1"
logger= logging.getLogger("OpenVisus")

# uncomment for debugging
# ov.SetupLogger(logger, output_stdout=True) 
print("OpenVisus imported")

# ///////////////////////////////////////////////////////////////
def ShowImage(img):
    """
    Utility to show an image using matplot lib
    """
    print("dtype",img.dtype,"shape",img.shape,"m",np.min(img),"M",np.max(img))
    fig = plt.figure()
    fig.set_size_inches(18.5, 10.5)

    ax = fig.add_subplot(1, 2, 1) # nrows, ncols, index
    imgplot = plt.imshow(img)

    plt.colorbar(orientation='horizontal')

    # sho historgram
    ax = fig.add_subplot(1, 2, 2)
    histogram, bin_edges = np.histogram(img, bins=256, range=(np.min(img), np.max(img)))
    plt.title("Histogram")
    plt.xlabel("value")
    plt.ylabel("pixel count")
    plt.xlim([np.min(img), np.max(img)])
    plt.plot(bin_edges[0:-1], histogram)


# Donwload a TIFF stack 

NOTE: **it's 4GB file** and will take a lot to download and unzip

In [ ]:
filenames=list(sorted(glob.glob("/mnt/data/chess/assets/image-stack/*.tif")))
if not filenames:

    from openvisuspy.utils import DownloadObject
    DownloadObject("s3://utah/assets/image-stack.zip", "/mnt/data/chess/assets/image-stack.zip")

    import zipfile
    with zipfile.ZipFile("/mnt/data/chess/assets/image-stack.zip", 'r') as zip_ref:
        zip_ref.extractall("/mnt/data/chess/assets/")
    filenames=list(sorted(glob.glob("/mnt/data/chess/assets/image-stack/*.tif")))

D=len(filenames)
assert(D)
img = io.imread(filenames[D//2])
H,W=img.shape
print("W",W,"H",H,"D",D,"dtype",img.dtype)
ShowImage(np.clip(img, 0, 70))

# Create the IDX file

In [ ]:
idx_filename="./remove-me/image-stack/visus.idx"

# ************ dangerous (make sure you are in a tmp directory) ************ 
import os,sys,shutil
assert('remove-me' in idx_filename)
shutil.rmtree(os.path.dirname(idx_filename), ignore_errors=True)

fields=[ov.Field("data",str(img.dtype),"row_major")]
db=ov.CreateIdx(
	url=idx_filename,
	dims=[W,H,D],
	fields=fields,
	compression="raw")

print(type(db))
print(db.getDatasetBody().toString())

# Write data to OpenVisus

In [ ]:
def generateSlices():
    for Z in range(D): 
        # print(f"Writing {Z}/{D}...")
        yield io.imread(filenames[Z])

t1 = time.time()
db.writeSlabs(generateSlices())
print(f"db.write done in {time.time() - t1} seconds")

# Show OpenVisus data

In [ ]:
Z=D//2
slice=db.read(x=[0,W],y=[0,H],z=[Z,Z+1], num_refinements=1)[0,:,:]
print(slice.shape,slice.dtype)
ShowImage(np.clip(slice, 0, 70))